Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [27]:
import numpy as np
from random import randint
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [28]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

def generate_instance(universe_size, num_sets, density):
    sets = np.random.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(sets[:, s]):
            sets[randint(0, num_sets - 1), s] = True
    costs = np.power(sets.sum(axis=1), 1.1)
    return sets, costs

In [29]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

KeyboardInterrupt: 

## Helper Functions

In [24]:
def valid(solution, sets):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(sets[solution]))


def cost(solution, costs):
    """Returns the cost of a solution (to be minimized)"""
    return costs[solution].sum()

def tweak(solution):
    """Generates a new solution by flipping one random set"""
    new_solution = solution.copy()
    index = randint(0, len(solution) - 1)
    new_solution[index] = not new_solution[index]
    return new_solution


## Have Fun!

## Hill Climbing Algorithm

In this section, we implement the **Hill Climbing** algorithm to solve the Set Cover problem. The algorithm starts with a random solution and iteratively improves it by tweaking to either increase coverage or decrease cost.

### Algorithm Steps:
1. Start with a random solution (50% of sets selected).
2. Calculate the coverage and cost of the current solution.
3. Generate a new solution by flipping a random set (tweak).
4. Accept the new solution if:
    - It increases the coverage of the universe, or
    - It maintains the same coverage but reduces the cost.
5. Repeat for a fixed number of iterations (`max_steps`).


In [25]:
def hill_climbing(sets, costs, max_steps=1000):
    solution = np.random.rand(sets.shape[0]) < 0.5  # Start with a random solution
    best_solution = solution
    best_coverage = np.sum(np.logical_or.reduce(sets[solution]))
    best_cost = cost(solution, costs)

    for step in range(max_steps):
        # Generate a new solution by tweaking the current one
        new_solution = tweak(solution)
        new_coverage = np.sum(np.logical_or.reduce(sets[new_solution]))

        # Accept the new solution if it improves coverage or cost
        if new_coverage > best_coverage or (new_coverage == best_coverage and cost(new_solution, costs) < best_cost):
            solution = new_solution
            best_coverage = new_coverage
            best_cost = cost(new_solution, costs)

    return best_solution, best_coverage, best_cost


## Main Execution

In this section, we define multiple problem instances from the Lab 1 instructions, varying the universe size, number of sets, and density. For each instance, we run the Hill Climbing algorithm and print the results for evaluation.


In [ ]:
# Lab 1 problem instances
instances = [
    {"universe_size": 100, "num_sets": 10, "density": 0.2},
    {"universe_size": 1000, "num_sets": 100, "density": 0.2},
    {"universe_size": 10000, "num_sets": 1000, "density": 0.2},
    {"universe_size": 100000, "num_sets": 10000, "density": 0.1},
    {"universe_size": 100000, "num_sets": 10000, "density": 0.2},
    {"universe_size": 100000, "num_sets": 10000, "density": 0.3}
]

# Execute Hill Climbing for all instances and print results
for instance in instances:
    universe_size = instance["universe_size"]
    num_sets = instance["num_sets"]
    density = instance["density"]

    print(f"Running for Universe Size: {universe_size}, Num Sets: {num_sets}, Density: {density}")

    # Generate the instance
    sets, costs = generate_instance(universe_size, num_sets, density)

    # Run Hill Climbing
    best_solution, best_coverage, best_cost = hill_climbing(sets, costs, max_steps=1000)

    # Print the results for evaluation
    print(f"Best Coverage: {best_coverage}/{universe_size}")
    print(f"Best Cost: {best_cost}\n")
